In [ ]:
#뉴스 크롤링

In [18]:
import requests as req
from bs4 import BeautifulSoup as bs

In [43]:
url = input()

https://sports.v.daum.net/v/20210324084734181


In [44]:
#url = 'https://sports.v.daum.net/v/20210324084734181'

res = req.get(url)
soup = bs(res.text, 'html.parser')

figcaptions = soup.select('figcaption')

for figcaption in figcaptions:
    figcaption.extract()

title = soup.select_one('h3.tit_view')
content = soup.select_one('div#harmonyContainer')

print('뉴스제목:',title.text)
print('내용:',content.text.strip())   #strip() 공백없애기

뉴스제목: '토트넘 초비상'..레알, '떠나고픈' 케인 영입 추진한다
내용: [인터풋볼] 김대식 기자 = 레알 마드리드가 해리 케인(27, 토트넘) 영입을 고민하고 있다.

스페인 '카네다 세르'는 23일(이하 한국시간) "킬리안 음바페(22, 파리생제르맹)는 레알의 최우선 타깃이다. 엘링 홀란드(20, 도르트문트)는 모든 유럽 빅클럽들이 지켜보고 있다. 하지만 레알은 3번째 타깃으로 프리미어리그(EPL) 득점 선두를 달리고 있는 해리 케인 역시 계속해서 관찰하고 있다"고 보도했다.
케인은 2014-15시즌부터 토트넘의 핵심 선수로 활약했다. 프리미어리그 득점왕만 2차례 차지했으며, 구단 역사상 최다 득점자 2위에 올랐을 정도로 꾸준했다. 최근에는 몇 차례 부상을 겪었지만 케인의 유무에 따라 토트넘의 경기력은 차이가 날 정도로 케인의 영향력은 절대적이다. 이번 시즌도 케인은 리그 27경기에서 17골 13도움을 기록하며 EPL득점과 도움 1위라는 엄청난 성적을 보여주고 있다. 손흥민과 함께 토트넘을 이끌어가고 있다고 해도 과언이 아니다.
레알이 진지하게 케인을 노리고 있다면 선수는 이적을 원할 수도 있다. 미국 '디 애슬리틱' 등에서 활동하며 유럽 이적시장 전문가로 유명한 데이비드 온스테인 기자는 "케인은 공개적으로 언급하지는 않겠지만 그는 토트넘을 떠나고 싶어한다. 그는 주장이고, 팀에 헌신적이지만 확실히 트로피를 원한다. 많은 우승을 꿈꾸고 있다. 토트넘을 떠나는 데 열려있다"고 분석했기 때문이다.
역시 걸림돌은 이적료다. 스테인 기자는 "케인이 떠나고 싶어도 토트넘을 떠나기는 쉽지 않다. 다니엘 레비 회장은 엄청난 이적료를 요구할 가능성이 높기 때문이다. 최소 1억 2000만 파운드(약 1888억 원) 이상의 제의가 와야 이적을 고려할 것"이라고 분석했다. '카네다 세르'도 "레알은 가레스 베일의 토트넘 임대가 이번 거래의 도움이 될 수 있다고 믿는다. 그래도 케인을 데려오기 위해선 상당한 이적료를 지불해야 할 것"이라고 전망했다.
음바페의 잔류 가능성이 

In [20]:
#뉴스 내용 요약

In [21]:
from gensim.summarization.summarizer import summarize

In [22]:
print(summarize(content.text))

미국 '디 애슬리틱' 등에서 활동하며 유럽 이적시장 전문가로 유명한 데이비드 온스테인 기자는 "케인은 공개적으로 언급하지는 않겠지만 그는 토트넘을 떠나고 싶어한다.
스테인 기자는 "케인이 떠나고 싶어도 토트넘을 떠나기는 쉽지 않다.
다니엘 레비 회장은 엄청난 이적료를 요구할 가능성이 높기 때문이다.
'카네다 세르'도 "레알은 가레스 베일의 토트넘 임대가 이번 거래의 도움이 될 수 있다고 믿는다.


In [23]:
import os
import re
import json
import string
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, TFBertModel

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import urllib
import wget

MAX_LEN = 384
EPOCHS = 3
VERBOSE = 2
BATCH_SIZE = 16

In [24]:
DATA_OUT_PATH = './data_out/KOR'

In [25]:
def plot_graphs(history, string, string_1, string_2):
    # loss 
    plt.plot(history.history[string])
    plt.plot(history.history[string_1])
    plt.plot(history.history[string_2])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, string_1, string_2])
    plt.show()

In [26]:
SEED_NUM = 1234
tf.random.set_seed(SEED_NUM)
np.random.seed(SEED_NUM)

In [27]:
# Save the slow pretrained tokenizer
slow_tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased", lowercase=False)
save_path = "bert-base-multilingual-cased/"
if not os.path.exists(save_path):
    os.makedirs(save_path)
slow_tokenizer.save_pretrained(save_path)

# Load the fast tokenizer from saved file
tokenizer = BertWordPieceTokenizer("bert-base-multilingual-cased/vocab.txt", lowercase=False)

In [28]:
train_data_url = "https://korquad.github.io/dataset/KorQuAD_v1.0_train.json"
train_path = keras.utils.get_file("train.json", train_data_url)
eval_data_url = "https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json"
eval_path = keras.utils.get_file("eval.json", eval_data_url)

In [85]:
wget.download('https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-config.json', out='./bert-base-multilingual-cased/')

100% [..................................................................................] 625 / 625

'./bert-base-multilingual-cased//bert-base-multilingual-cased-config.json'

In [91]:
os.rename('./bert-base-multilingual-cased/bert-base-multilingual-cased-config.json', './bert-base-multilingual-cased/config.json')

FileNotFoundError: [WinError 2] 지정된 파일을 찾을 수 없습니다: './bert-base-multilingual-cased/bert-base-multilingual-cased-config.json' -> './bert-base-multilingual-cased/config.json'

In [87]:
wget.download('https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-tf_model.h5', out='./bert-base-multilingual-cased/')

100% [....................................................................] 1083389348 / 1083389348

'./bert-base-multilingual-cased//bert-base-multilingual-cased-tf_model.h5'

In [90]:
os.rename('./bert-base-multilingual-cased/bert-base-multilingual-cased-tf_model.h5', './bert-base-multilingual-cased/tf_model.h5')

FileNotFoundError: [WinError 2] 지정된 파일을 찾을 수 없습니다: './bert-base-multilingual-cased/bert-base-multilingual-cased-tf_model.h5' -> './bert-base-multilingual-cased/tf_model.h5'

In [29]:
class SquadExample:
    def __init__(self, question, context, start_char_idx, answer_text):
        self.question = question
        self.context = context
        self.start_char_idx = start_char_idx
        self.answer_text = answer_text
        self.skip = False

    def preprocess(self):
        context = self.context
        question = self.question
        answer_text = self.answer_text
        start_char_idx = self.start_char_idx

        # Clean context, answer and question
        context = " ".join(str(context).split())
        question = " ".join(str(question).split())
        answer = " ".join(str(answer_text).split())

        # Find end character index of answer in context
        end_char_idx = start_char_idx + len(answer)
        if end_char_idx >= len(context):
            self.skip = True
            return

        # Mark the character indexes in context that are in answer
        is_char_in_ans = [0] * len(context)
        for idx in range(start_char_idx, end_char_idx):
            is_char_in_ans[idx] = 1

        # Tokenize context
        tokenized_context = tokenizer.encode(context)

        # Find tokens that were created from answer characters
        ans_token_idx = []
        for idx, (start, end) in enumerate(tokenized_context.offsets):
            if sum(is_char_in_ans[start:end]) > 0:
                ans_token_idx.append(idx)

        if len(ans_token_idx) == 0:
            self.skip = True
            return

        # Find start and end token index for tokens from answer
        start_token_idx = ans_token_idx[0]
        end_token_idx = ans_token_idx[-1]

        # Tokenize question
        tokenized_question = tokenizer.encode(question)

        # Create inputs
        input_ids = tokenized_context.ids + tokenized_question.ids[1:]
        token_type_ids = [0] * len(tokenized_context.ids) + [1] * len(
            tokenized_question.ids[1:]
        )
        attention_mask = [1] * len(input_ids)

        # Pad and create attention masks.
        # Skip if truncation is needed
        padding_length = MAX_LEN - len(input_ids)
        if padding_length > 0:  # pad
            input_ids = input_ids + ([0] * padding_length)
            attention_mask = attention_mask + ([0] * padding_length)
            token_type_ids = token_type_ids + ([0] * padding_length)
        elif padding_length < 0:  # skip
            self.skip = True
            return

        self.input_ids = input_ids
        self.token_type_ids = token_type_ids
        self.attention_mask = attention_mask
        self.start_token_idx = start_token_idx
        self.end_token_idx = end_token_idx
        self.context_token_to_char = tokenized_context.offsets


def create_squad_examples(raw_data):
    squad_examples = []
    for item in raw_data["data"]:
        for para in item["paragraphs"]:
            context = para["context"]
            for qa in para["qas"]:
                question = qa["question"]
                answer_text = qa["answers"][0]["text"]
                start_char_idx = qa["answers"][0]["answer_start"]
                squad_eg = SquadExample(
                    question, context, start_char_idx, answer_text
                )
                squad_eg.preprocess()
                squad_examples.append(squad_eg)
    return squad_examples


def create_inputs_targets(squad_examples):
    dataset_dict = {
        "input_ids": [],
        "token_type_ids": [],
        "attention_mask": [],
        "start_token_idx": [],
        "end_token_idx": [],
    }
    for item in squad_examples:
        if item.skip == False:
            for key in dataset_dict:
                dataset_dict[key].append(getattr(item, key))
    for key in dataset_dict:
        dataset_dict[key] = np.array(dataset_dict[key])

    x = [
        dataset_dict["input_ids"],
        dataset_dict["token_type_ids"],
        dataset_dict["attention_mask"],
    ]
    y = [dataset_dict["start_token_idx"], dataset_dict["end_token_idx"]]
    return x, y


In [30]:
with open(train_path) as f:
    raw_train_data = json.load(f)

with open(eval_path) as f:
    raw_eval_data = json.load(f)


train_squad_examples = create_squad_examples(raw_train_data)
x_train, y_train = create_inputs_targets(train_squad_examples)
print(f"{len(train_squad_examples)} training points created.")

eval_squad_examples = create_squad_examples(raw_eval_data)
x_eval, y_eval = create_inputs_targets(eval_squad_examples)
print(f"{len(eval_squad_examples)} evaluation points created.")

60407 training points created.
5774 evaluation points created.


In [31]:
class TFBERTQuestionAnswering(tf.keras.Model):
    def __init__(self, model_name, dir_path, num_class):
        super(TFBERTQuestionAnswering, self).__init__()
        
        self.encoder = TFBertModel.from_pretrained(model_name, cache_dir=dir_path)
        self.start_logit = tf.keras.layers.Dense(num_class, name="start_logit", use_bias=False)
        self.end_logit = tf.keras.layers.Dense(num_class, name="end_logit", use_bias=False)
        self.flatten = tf.keras.layers.Flatten() 
        self.softmax = tf.keras.layers.Activation(tf.keras.activations.softmax)
        
    def call(self, inputs):
        input_ids, token_type_ids, attention_mask = inputs
        embedding = self.encoder(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)[0]
        start_logits = self.start_logit(embedding)
        start_logits = self.flatten(start_logits)
        
        end_logits = self.end_logit(embedding)
        end_logits = self.flatten(end_logits)
        
        start_probs = self.softmax(start_logits)
        end_probs = self.softmax(end_logits)
    
        return start_probs, end_probs

In [32]:
korquad_model = TFBERTQuestionAnswering(model_name='./bert-base-multilingual-cased/',dir_path='bert_ckpt', num_class=1)
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False)

Some layers from the model checkpoint at ./bert-base-multilingual-cased/ were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at ./bert-base-multilingual-cased/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [33]:
def normalized_answer(s):    
    def remove_(text):
        ''' 불필요한 기호 제거 '''
        text = re.sub("'", " ", text)
        text = re.sub('"', " ", text)
        text = re.sub('《', " ", text)
        text = re.sub('》', " ", text)
        text = re.sub('<', " ", text)
        text = re.sub('>', " ", text) 
        text = re.sub('〈', " ", text)
        text = re.sub('〉', " ", text)   
        text = re.sub("\(", " ", text)
        text = re.sub("\)", " ", text)
        text = re.sub("‘", " ", text)
        text = re.sub("’", " ", text)      
        return text

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_punc(lower(remove_(s))))

In [34]:
class ExactMatch(keras.callbacks.Callback):
    def __init__(self, x_eval, y_eval):
        self.x_eval = x_eval
        self.y_eval = y_eval

    def on_epoch_end(self, epoch, logs=None):
        pred_start, pred_end = self.model.predict(self.x_eval)
        count = 0
        eval_examples_no_skip = [_ for _ in eval_squad_examples if _.skip == False]
        for idx, (start, end) in enumerate(zip(pred_start, pred_end)):
            squad_eg = eval_examples_no_skip[idx]
            offsets = squad_eg.context_token_to_char
            start = np.argmax(start)
            end = np.argmax(end)
            if start >= len(offsets):
                continue
            pred_char_start = offsets[start][0]
            if end < len(offsets):
                pred_char_end = offsets[end][1]
                pred_ans = squad_eg.context[pred_char_start:pred_char_end]
            else:
                pred_ans = squad_eg.context[pred_char_start:]

            normalized_pred_ans = normalized_answer(pred_ans)
            normalized_true_ans = normalized_answer(squad_eg.answer_text)
            if normalized_pred_ans in normalized_true_ans:
                count += 1
        acc = count / len(self.y_eval[0])
        print(f"\nepoch={epoch+1}, exact match score={acc:.2f}")

In [35]:
exact_match_callback = ExactMatch(x_eval, y_eval)

In [36]:
print(tokenizer.decode(x_eval[0][5]))
#print(tokenizer.decode(y_eval[0][0]))


1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의 ( 폭력행위등처벌에관한법률위반 ) 으로 지명수배되었다. 1989년 3월 12일 서울지방검찰청 공안부는 임종석의 사전구속영장을 발부받았다. 같은 해 6월 30일 평양축전에 임수경을 대표로 파견하여 국가보안법위반 혐의가 추가되었다. 경찰은 12월 18일 ~ 20일 사이 서울 경희대학교에서 임종석이 성명 발표를 추진하고 있다는 첩보를 입수했고, 12월 18일 오전 7시 40분 경 가스총과 전자봉으로 무장한 특공조 및 대공과 직원 12명 등 22명의 사복 경찰을 승용차 8대에 나누어 경희대학교에 투입했다. 1989년 12월 18일 오전 8시 15분 경 서울청량리경찰서는 호위 학생 5명과 함께 경희대학교 학생회관 건물 계단을 내려오는 임종석을 발견, 검거해 구속을 집행했다. 임종석은 청량리경찰서에서 약 1시간 동안 조사를 받은 뒤 오전 9시 50분 경 서울 장안동의 서울지방경찰청 공안분실로 인계되었다. 1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 사람의 이름은?


In [37]:
korquad_model.compile(optimizer=optimizer, loss=[loss, loss])

In [38]:
model_name = "tf2_bert_korquad"

checkpoint_path = os.path.join(DATA_OUT_PATH, model_name, 'weights.h5')
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create path if exists
if os.path.exists(checkpoint_dir):
    print("{} -- Folder already exists \n".format(checkpoint_dir))
else:
    os.makedirs(checkpoint_dir, exist_ok=True)
    print("{} -- Folder create complete \n".format(checkpoint_dir))
    
cp_callback = ModelCheckpoint(
    checkpoint_path, verbose=1, save_best_only=True, save_weights_only=True)

./data_out/KOR\tf2_bert_korquad -- Folder already exists 



In [39]:
history = korquad_model.fit(
    x_train,
    y_train,
    epochs=EPOCHS,  # For demonstration, 3 epochs are recommended
    verbose=VERBOSE,
    batch_size=BATCH_SIZE,
    callbacks=[exact_match_callback, cp_callback]
)

Epoch 1/3
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


KeyboardInterrupt: 

In [40]:
korquad_model.load_weights('./BERT/weights_KorQuAD.h5')
#korquad_model.load_weights("weights_KorQuAD.h5")


In [41]:
korquad_model.loss

ListWrapper([<tensorflow.python.keras.losses.SparseCategoricalCrossentropy object at 0x0000023ED4B8D310>, <tensorflow.python.keras.losses.SparseCategoricalCrossentropy object at 0x0000023ED4B8D310>])

In [42]:
class SquadExample_pred:
    def __init__(self, question, context):#, start_char_idx, answer_text):
        self.question = question
        self.context = context
        self.skip = False

    def preprocess(self):
        context = self.context
        question = self.question
        #answer_text = self.answer_text
        #start_char_idx = self.start_char_idx

        # Clean context, answer and question
        context = " ".join(str(context).split())
        question = " ".join(str(question).split())

        # Tokenize context
        tokenized_context = tokenizer.encode(context)

        # Tokenize question
        tokenized_question = tokenizer.encode(question)

        # Create inputs
        input_ids = tokenized_context.ids + tokenized_question.ids[1:]
        token_type_ids = [0] * len(tokenized_context.ids) + [1] * len(
            tokenized_question.ids[1:]
        )
        attention_mask = [1] * len(input_ids)
        

        # Pad and create attention masks.
        # Skip if truncation is needed
        padding_length = MAX_LEN - len(input_ids)
        if padding_length > 0:  # pad
            input_ids = input_ids + ([0] * padding_length)
            attention_mask = attention_mask + ([0] * padding_length)
            token_type_ids = token_type_ids + ([0] * padding_length)
        elif padding_length < 0:  # skip
            self.skip = True
            return

        self.input_ids = input_ids
        self.token_type_ids = token_type_ids
        self.attention_mask = attention_mask
        self.context_token_to_char = tokenized_context.offsets
        #해당 코드의 의미를 모르겠습니다...
        
    def get_input_target(self):
        dataset_dict = {
            "input_ids": [],
            "token_type_ids": [],
            "attention_mask": [],
        }
        if self.skip == False:
            for key in dataset_dict:
                dataset_dict[key].append(getattr(self, key))
        for key in dataset_dict:
            dataset_dict[key] = np.array(dataset_dict[key])

        x = [
            dataset_dict["input_ids"],
            dataset_dict["token_type_ids"],
            dataset_dict["attention_mask"],
        ]
        return x

def create_squad_examples_from_arg(question, context):#, start_char_idx, answer_text):
    squad_eg = SquadExample_pred(
        question, context#, start_char_idx, answer_text
    )
    squad_eg.preprocess()
    return squad_eg


def predict_test(model, pred_raw):
    x_pred = pred_raw.get_input_target()
    pred_start, pred_end = model.predict(x_pred)
    
    pred_start_offset_index = np.argmax(pred_start)
    pred_end_offset_index = np.argmax(pred_end)
    pred_start_offset = pred_raw.context_token_to_char[pred_start_offset_index]
    pred_end_offset = pred_raw.context_token_to_char[pred_end_offset_index]
    answer = pred_context[pred_start_offset[0]:pred_end_offset[1]]
    
    normalized_pred_ans = normalized_answer(answer)
    
    return normalized_pred_ans


#pred_question = "천문기기를 제작한 공을 인정받아 면천된 년도는?"
#pred_context = "장영실은 발명가로서의 훌륭한 재주를 태종이 인정하여 발탁하였다. 세종은 실용주의자로서 장영실의 적성을 중요하게 생각하여 부왕의 뒤를 이어 중용하였다. 세종은 장영실을 1421년(세종 4년) 윤사웅(尹士雄), 최천구(崔天衢) 등과 함께 중국에 보내어 천문기기의 모양을 배워오도록 했다. 귀국 후 장영실 나이 약 34세 때인 1423년(세종 5년)에 천문기기를 제작한 공을 인정받아 면천되었고 다시 대신들의 의논을 거쳐 종5품 상의원(尙衣院) 별좌에 임명되었다.[3] 1424년(세종6) 5월, 임금은 그를 정5품 행사직으로 승진시켰고 갱점지기(更点之器)를 만들라고 명했다. 그래서 나온 것이 첫 물시계였다."

pred_question = "득점왕만 두 차례 차지한?"
pred_context = content.text[0:384]
pred_data = create_squad_examples_from_arg(pred_question, pred_context)
pred_answer = predict_test(korquad_model, pred_data)

print(pred_context)
print("\n")
print("Answer : " + pred_answer)













[인터풋볼] 김대식 기자 = 레알 마드리드가 해리 케인(27, 토트넘) 영입을 고민하고 있다.

스페인 '카네다 세르'는 23일(이하 한국시간) "킬리안 음바페(22, 파리생제르맹)는 레알의 최우선 타깃이다. 엘링 홀란드(20, 도르트문트)는 모든 유럽 빅클럽들이 지켜보고 있다. 하지만 레알은 3번째 타깃으로 프리미어리그(EPL) 득점 선두를 달리고 있는 해리 케인 역시 계속해서 관찰하고 있다"고 보도했다.
케인은 2014-15시즌부터 토트넘의 핵심 선수로 활약했다. 프리미어리그 득점왕만 2차례 차지했으며, 구단 역사상 최다 득점자 2위에 올랐을 정도로 꾸준했다. 최근에는 몇 차례 부상을 겪었지만 케인의 유무에 따라 토트넘의 경기력은 차이가 날 정도로 케인의


Answer : 프리
